# Clasificador de Imágenes: Perros vs Gatos

## Descripción
Este proyecto desarrolla un modelo de Deep Learning basado en Redes Neuronales Convolucionales (CNN) para clasificar imágenes de perros y gatos.

## Tecnologías utilizadas
- Python
- TensorFlow / Keras
- Jupyter Notebook

## 1. Importación de librerías

In [ ]:
from keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.losses import BinaryCrossentropy
from keras.optimizers import Adam
from keras.utils import PyDataset
import imgaug.augmenters as iaa
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import os
import cv2

## 2. Definición de rutas y etiquetas


In [ ]:
IMG_DIR = "cats_and_dogs"  
IMAGES = [os.path.join(IMG_DIR, "cats"), os.path.join(IMG_DIR, "dogs")]# recoge las dos carpetas
LABELS = [0, 1]  #etiqueta



## 3. Creación del DataFrame


In [ ]:

all_images = []

for img_dir, label in zip(IMAGES, LABELS):
    for filename in os.listdir(img_dir):
        if filename.lower().endswith((".jpg", ".png", ".jpeg")):
            img_path = os.path.join(img_dir, filename)#ruta de archivo
            all_images.append((img_path, label))


df = pd.DataFrame(all_images, columns=["img_path", "label"])
df = df.sample(frac=1).reset_index(drop=True)

## 4. Configuración y Data Augmentation



In [ ]:

BATCH_SIZE = 32
SIZE = 128

# Pipeline para entrenamiento
train_seq = iaa.Sequential([
    iaa.CropToFixedSize(height=90, width=90),
    iaa.Fliplr(0.5),
    iaa.Sometimes(0.3, iaa.GaussianBlur(sigma=(0.0, 2.0))),
    iaa.Sometimes(0.4, iaa.Affine(rotate=(-20, 20))),
    iaa.Resize({"height": SIZE, "width": SIZE})
])

# Pipeline para test
test_seq = iaa.Sequential([
    iaa.Resize({"height": SIZE, "width": SIZE})
])



## 5. Función de carga y preprocesamiento de imágenes


In [ ]:

def get_data(img_path, augmentor=None):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if augmentor is not None:
        img = augmentor(image=img)
    return img


## 6. Creación del Dataset personalizado


In [ ]:

class CatDogDataset(PyDataset):
    def __init__(self, image_paths, labels, aug, batch_size=BATCH_SIZE, train= True, **kwargs):
        super().__init__(**kwargs)
        self.image_paths = image_paths
        self.labels = labels
        self.aug = aug
        self.batch_size = batch_size
        self.train = train
        self.on_epoch_end()
    
    def __len__(self):
        return len(self.image_paths) // self.batch_size
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.image_paths)) 
        if self.train:
            np.random.shuffle(self.indexes)
    
    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        image_paths_temp = [self.image_paths[k] for k in indexes]
        labels_temp = [self.labels[k] for k in indexes]

        # para entrenar
        images = np.array([get_data(p, self.aug)/255.0 for p in image_paths_temp], dtype=np.float32)
        labels = np.array(labels_temp, dtype=np.float32)

        return images, labels

## 7. División del conjunto de datos


In [ ]:
## DIVISIÓN TRAIN / TEST
samples = df["img_path"].tolist()
labels = df["label"].tolist()   

split = int(len(samples) * 0.8)

# Datasets con la augmentación
train_dataset = CatDogDataset(samples[:split], labels[:split], train_seq, train=True)
test_dataset = CatDogDataset(samples[split:], labels[split:], test_seq, train=False)

## 8. Construcción del modelo CNN


Se construye una red neuronal convolucional (CNN) simple para clasificación binaria (perros vs gatos).


In [ ]:
CHANNELS = 3  
NUM_CLASSES = 1  

optimizer = Adam()

model = Sequential()
model.add(Input(shape = (SIZE, SIZE, CHANNELS)))

model.add(Conv2D(filters=32, kernel_size=(3,3), activation="relu"))# Bloques convencionales
model.add(Conv2D(filters=32, kernel_size=(3,3), activation="relu"))# Bloques convencionales
model.add(MaxPooling2D(2)) #model.add(MaxPooling2D(pool_size=(2,2))) Bloques convencionales

model.add(Flatten()) #capas densas profundas
model.add(Dense(units = 128, activation = "relu"))# capas densas profundas
model.add(Dense(units = 64, activation = "relu"))
model.add(Dense(units = 32, activation = "relu"))

model.add(Dense(units=NUM_CLASSES, activation="sigmoid")) # (gato/perro)

# Compilación
model.compile(
    loss=BinaryCrossentropy(),
    optimizer=optimizer,
    metrics=['binary_accuracy']
)

model.summary()

## 9. Entrenamiento del modelo


In [ ]:

EPOCHS = 20

history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=EPOCHS)



## 10. Evaluación y visualización de resultados


In [ ]:
plt.figure(figsize=(6,4))

plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")

plt.title("Evolución de la Loss")
plt.xlabel("Épocas")
plt.ylabel("Loss")
plt.legend()

plt.show()


In [ ]:
plt.figure(figsize=(6,4))

plt.plot(history.history["binary_accuracy"], label="Train Accuracy")
plt.plot(history.history["val_binary_accuracy"], label="Validation Accuracy")

plt.title("Evolución de la Accuracy")
plt.xlabel("Épocas")
plt.ylabel("Accuracy")
plt.legend()

plt.show()


## 11. Análisis de resultados

El modelo presenta una pérdida proxima a 0.693 y una precisión alrededor del 50%, lo que indica que su rendimiento es equivalente a una clasificación aleatoria.

Este comportamiento sugiere que el modelo no está aprendiendo patrones relevantes en los datos.

Posibles causas:
- Arquitectura demasiado simple
- Preprocesamiento inadecuado
- Data augmentation excesivo
- Número insuficiente de épocas




## 12. Primera Iteración

Se realizó una modificación en el pipeline de augmentación sustituyendo `CropToFixedSize` por `CenterCropToFixedSize`, con el objetivo de preservar mayor información relevante de la imagen.

Tras la modificación se observó:
- Mejora en la precisión de entrenamiento
- Peor desempeño en validación

Este comportamiento sugiere posible sobreajuste (overfitting).


In [ ]:

# Pipeline cambiado 
train_seq_iter1 = iaa.Sequential([
    iaa.CenterCropToFixedSize(height=90, width=90),# cambio para volver a entrenar
    iaa.Fliplr(0.5),
    iaa.Sometimes(0.3, iaa.GaussianBlur(sigma=(0.0, 2.0))),
    iaa.Sometimes(0.4, iaa.Affine(rotate=(-20, 20))),
    iaa.Resize({"height": SIZE, "width": SIZE})
])

train_dataset_iter1 = CatDogDataset(samples[:split], labels[:split], train_seq_iter1, train=True)
test_dataset_iter1  = CatDogDataset(samples[split:], labels[split:], test_seq_iter1, train=False)

EPOCHS = 25

history_iter1 = model.fit(
    train_dataset_iter1,
    validation_data=test_dataset_iter1,
    epochs=EPOCHS
)

| Modelo        | Train Accuracy | Validation Accuracy |
|--------------|---------------|---------------------|
| Original     | 0.49          | 0.48                |
| Iteración 1  | 0.60          | 0.50                |

